In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fastai.vision.all import *
from utils import *
import sklearn.metrics as skm
from tqdm.notebook import tqdm

In [3]:
conf = {
    'bs':180,
    'arch':resnet34,
    'split':0
}

In [4]:
train_df = pd.read_csv(PATH/'train_split.csv')
train_df['is_valid'] = train_df.split==conf['split']

In [5]:
# def hash_label(x):
#     x = (13*x)%10000
#     return x // 2000
# train_df['split']=train_df.label_group.apply(hash_label)
# train_df.to_csv(PATH/'train_split.csv')

In [6]:
data_block = DataBlock(blocks = (ImageBlock(), CategoryBlock(vocab=train_df.label_group.to_list())),
                 splitter=ColSplitter(),
                 #splitter=RandomSplitter(),
                 get_y=ColReader('label_group'),
                 get_x=ColReader('image', pref=PATH/'train_images'),
                 item_tfms=Resize(460),
                 batch_tfms=aug_transforms(size=224, min_scale=0.75),
                 )
dls = data_block.dataloaders(train_df, bs=conf['bs'],num_workers=16)

In [7]:
#dls.valid.show_batch(max_n=16)

In [8]:
def emb_sim(inp):
    normal_inp=F.normalize(inp)
    return normal_inp @ normal_inp.T

In [9]:
def groups(preds):
    matches = preds[:,None]==preds[None,:]
    groups = [torch.where(matches[i])[0] for i in range(len(matches))]
    return groups
groups(torch.tensor([1,2,2,2,3,1]))

[tensor([0, 5]),
 tensor([1, 2, 3]),
 tensor([1, 2, 3]),
 tensor([1, 2, 3]),
 tensor([4]),
 tensor([0, 5])]

In [10]:
def f1_score(pred_groups, real_groups):
    res =0
    for pred, real in zip(pred_groups, real_groups):
        #print(pred, real)
        inter = set(pred.tolist()).intersection(set(real.tolist()))
        #print(inter)
        res += 2 * len(inter) / (len(pred)+len(real))
    return res / len(pred_groups)

In [28]:
def groups_from_embs(x, treshold=0.9, sims=None):
    if sims is None: sims = emb_sim(x)
    preds_groups=[]
    for i in range(len(sims)):
        tops= torch.where(sims[i]>treshold)[0]
        preds_groups.append(tops)
    return preds_groups

In [29]:
def embs_from_resnet(model, dl):
    model_stem =list(learn.model.children())[0].eval()
    pipeline = nn.Sequential(model_stem, AdaptiveConcatPool2d(), Flatten())
    all_embs = []
    all_ys=[]
    for batch in tqdm(dl):
        if len(batch) ==2:
            bx,by=batch
        else:
            bx,=batch
            by=torch.zeros(1)
        with torch.no_grad():
            embs = pipeline(bx)
            all_embs.append(embs.half())
        all_ys.append(by)
    return torch.cat(all_embs), torch.cat(all_ys)

In [13]:
def f1_from_preds(inp, tar):
    preds = inp.argmax(dim=1)
    pred_groups = groups(preds)
    real_groups = groups(tar)
    return f1_score(pred_groups, real_groups)

In [36]:
def treshold_finder(embs, y):
    real_groups = groups(y)
    sims = emb_sim(embs)
    scores = []
    ts =[]
    best, bestt=0,0
    for t in np.arange(0.8,1,.01):
        gr = groups_from_embs(x, t, sims)
        score = f1_score(gr, real_groups)
        scores.append(score)
        ts.append(t)
        if score > best: best,bestt = score, t
    plt.title(f'Best is {best:.3f} at {bestt:.3f}')
    plt.plot(ts, scores)
    plt.show()

In [14]:
learn = cnn_learner(dls,conf['arch'], metrics=[AccumMetric(f1_from_preds, flatten=False)])

In [15]:
learn.fine_tune(10,1e-2)

epoch,train_loss,valid_loss,f1_from_preds,time
0,7.969961,10.474304,0.276842,01:04


epoch,train_loss,valid_loss,f1_from_preds,time
0,5.536784,10.209629,0.348417,01:13
1,4.518547,10.830723,0.374114,01:14
2,3.777341,12.219127,0.416856,01:13
3,2.730000,13.326238,0.467988,01:13
4,1.817720,14.110039,0.499478,01:12
5,1.104580,14.515482,0.538400,01:13
6,0.618735,14.976788,0.550706,01:13
7,0.336454,15.163069,0.567583,01:13
8,0.188864,15.316378,0.578714,01:12
9,0.142370,15.351284,0.576094,01:13


In [ ]:
learn.save('resnet34')

# VALIDATION STATS

In [16]:
%%time
embs, y = embs_from_resnet(learn.model, dls.valid)

  0%|          | 0/38 [00:00<?, ?it/s]

CPU times: user 4.66 s, sys: 1.39 s, total: 6.06 s
Wall time: 11 s


In [17]:
%%time
x,y=learn.get_preds()

CPU times: user 6.23 s, sys: 1.89 s, total: 8.12 s
Wall time: 11.7 s


In [18]:
f1_from_preds(x,y)

0.5760942774944458

In [21]:
real_groups=groups(y)

In [22]:
emb_groups = groups_from_embs(embs,0.88)
emb_groups[:5]

[TensorBase([0], device='cuda:0'),
 TensorBase([1], device='cuda:0'),
 TensorBase([   2, 2684, 4807], device='cuda:0'),
 TensorBase([3], device='cuda:0'),
 TensorBase([   4, 5154], device='cuda:0')]

In [23]:
f1_score(emb_groups, real_groups)

0.7577581012917319

In [ ]:
treshold_finder(embs, y)

# Test 

In [43]:
fake_test_df = train_df[['posting_id', 'image', 'image_phash', 'title']]

In [44]:
fake_test_df = pd.concat([fake_test_df, fake_test_df])

In [45]:
fake_test_df

,posting_id,image,image_phash,title
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE"
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml
...,...,...,...,...
34245,train_4028265689,fff1c07ceefc2c970a7964cfb81981c5.jpg,e3cd72389f248f21,Masker Bahan Kain Spunbond Non Woven 75 gsm 3 ply lapis Bisa Dicuci
34246,train_769054909,fff401691371bdcb382a0d9075dfea6a.jpg,be86851f72e2853c,MamyPoko Pants Royal Soft - S 70 - Popok Celana
34247,train_614977732,fff421b78fa7284284724baf249f522e.jpg,ad27f0d08c0fcbf0,KHANZAACC Robot RE101S 1.2mm Subwoofer Bass Metal Wired Headset
34248,train_3630949769,fff51b87916dbfb6d0f8faa01bee67b8.jpg,e3b13bd1d896c05c,"Kaldu NON MSG HALAL Mama Kamu Ayam Kampung , Sapi Lokal, Jamur (Bkn Alsultan / Biocell)"


In [46]:
test_dl = dls.test_dl(fake_test_df)

In [64]:
embs, _ = embs_from_resnet(learn.model, test_dl)

  0%|          | 0/381 [00:00<?, ?it/s]

In [65]:
embs.shape

torch.Size([68500, 1024])

In [66]:
emb_groups = groups_from_embs(embs)


In [81]:
def matches(group, data):
    return ' '.join([data.iloc[x.cpu().item()].posting_id for x in group])

In [82]:
matches(emb_groups[1], fake_test_df)

'train_3386243561 train_1816968361 train_2120597446 train_3423213080 train_3805508898 train_3386243561 train_1816968361 train_2120597446 train_3423213080 train_3805508898'

In [97]:
%%time
m = [matches(g, fake_test_df) for g in emb_groups]

CPU times: user 2.25 s, sys: 10.5 ms, total: 2.26 s
Wall time: 2.26 s


In [85]:
fake_test_df['matches']=m

In [86]:
fake_test_df.head()

,posting_id,image,image_phash,title,matches
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,train_129225211 train_129225211
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE",train_3386243561 train_1816968361 train_2120597446 train_3423213080 train_3805508898 train_3386243561 train_1816968361 train_2120597446 train_3423213080 train_3805508898
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,train_2288590299 train_2288590299
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi,train_2406599165 train_1593362411 train_2406599165 train_1593362411
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,train_3369186413 train_921438619 train_3369186413 train_921438619
